In [1]:
from database_handling import DBHandling

In [2]:
# Utilisation de gradio pour l'app avec SQLite + Docker ? https://www.youtube.com/watch?v=-GgG1Wb2F64&list=PLi0b6yHwHZcGaIy4ZGkYLZqssvzwqkBia&index=19&ab_channel=MASTER2SISEDATASCIENCE
# https://www.sqlite.org/docs.html
# Avantage -> Juste un pacakge, pas besoin d'installer un serveur, + facile pour déploiement docker
# Acces avec des fichiers et des tables à l'intérieurs (pysqlite doit être installé avec C++, CMake etc -> Trop compliqué avec docker, on va utiliser pysqlite3 wheels -> version déjà compilée)
# Compatbile avec streamlit mais démo avec gradio
# Design pattern https://www.youtube.com/watch?v=BJatgOiiht4&ab_channel=ForrestKnight
## TODO faire le sqlite + ajouter des champs dans le scrapping(clef étrangère)
"""
- Reste Insertion Restaurants + Avis
- Delete Restaurants + Avis
- Update Restaurant + Avis
- Requêtes particulières ( restaurants avec plus de 4 étoilse etc...)

"""
# Changer la BDD pour chaque arro,dissement depuiss le champ adress
# Créer une table arrondissement

'\n- Reste Insertion Restaurants + Avis\n- Delete Restaurants + Avis\n- Update Restaurant + Avis\n- Requêtes particulières ( restaurants avec plus de 4 étoilse etc...)\n\n'

In [3]:
db = DBHandling("../../data/database.db")
# connection et création des tables
db.connect()
db.create_tables()
# help(db)


Connexion à la base de données réussie : ../../data/database.db
Tables créées avec succès.


In [4]:
# test selection
rows = db.fetch_all("SELECT * FROM restaurants")
rows

[]

In [5]:
import os
import pandas as pd

###################################### RUN ONLY ONCE ######################################

data_directory = "../../data"
description_directory = "../../data/descriptions"
all_restaurants = []
all_descriptions = []

# Read restaurant info CSV files
for filename in os.listdir(data_directory):
    filepath = os.path.join(data_directory, filename)

    if filename.endswith("_infos.csv"):
        try:
            df = pd.read_csv(filepath, encoding='utf-8', usecols=["Nom", "Type_Cuisine", "Fourchette_prix", "Adresse", "Note_moyenne"], index_col=False)
            all_restaurants.append(df)
        except pd.errors.EmptyDataError:
            print(f"Le fichier {filename} est vide")
        except Exception as e:
            print(f"Erreur lors du traitement de {filename}: {e}")

# Read description CSV files
for filename in os.listdir(description_directory):
    filepath = os.path.join(description_directory, filename)

    if filename.endswith("_description.csv"):
        try:
            df = pd.read_csv(filepath, encoding='utf-8', usecols=["Nom", "Description"], index_col=False)
            all_descriptions.append(df)
        except pd.errors.EmptyDataError:
            print(f"Le fichier {filename} est vide")
        except Exception as e:
            print(f"Erreur lors du traitement de {filename}: {e}")

# Concatenate all restaurant info DataFrames
if all_restaurants:
    all_restaurants_df = pd.concat(all_restaurants, ignore_index=True)

# Concatenate all description DataFrames
if all_descriptions:
    all_descriptions_df = pd.concat(all_descriptions, ignore_index=True)


In [6]:
all_restaurants_df.head(1)

,Nom,Type_Cuisine,Fourchette_prix,Adresse,Note_moyenne
0,L'affreux Jojo,"Italienne, Méditerranéenne, Saine","9,00€-15,00€","61 Rue de la Part-Dieu, 69003 Lyon France","5,0"


In [7]:
all_descriptions_df.head(1)

,Nom,Description
0,L'affreux Jojo,"Restaurant italien cosy, spécialités : panozzo..."


In [8]:
if not all_restaurants_df.empty and not all_descriptions_df.empty:
    merged_df = pd.merge(all_restaurants_df, all_descriptions_df, on="Nom", how="left")

    # Insert each row of the merged DataFrame into the database
    for _, row in merged_df.iterrows():
        try:
            db.insert_restaurant(
                nom=row["Nom"],
                type_cuisine=row["Type_Cuisine"],
                fourchette_prix=row["Fourchette_prix"],
                adresse=row["Adresse"],
                note_moyenne=row["Note_moyenne"],
                description=row["Description"]
            )
            print(f"Restaurant ajouté : {row['Nom']}")
        except KeyError as e:
            print(f"Erreur dans le fichier {filename}: colonne manquante {e}")
       

61, Rue de la Part-Dieu, Préfecture, Lyon 3e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69003, France
Insertion réussie.
Restaurant ajouté : L'affreux Jojo
132, Rue Bugeaud, Les Brotteaux, Lyon 6e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69006, France
Insertion réussie.
Restaurant ajouté : L'Argot
Rue d'Austerlitz, Gros Caillou, Lyon 4e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69004, France
Insertion réussie.
Restaurant ajouté : Bistrot Orcia
None
Restaurant Paul Bocuse
Paul Bocuse, 40, Rue de la Plage, Collonges-au-Mont-d'Or, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69660, France
Insertion réussie.
Restaurant ajouté : Restaurant Paul Bocuse
1, Quai du Commerce, L’Industrie, Vaise, Lyon 9e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69009, France
Inse

In [9]:
# # Parcourir les fichiers CSV dans le répertoire
# import os
# import csv

# ###################################### RUN ONLY ONCE ######################################

# data_directory = "../../data"
# for filename in os.listdir(data_directory):
#     filepath = os.path.join(data_directory, filename)

#     if filename.endswith("_infos.csv"):
#         # Traitement des fichiers contenant les informations des restaurants
#         with open(filepath, newline='', encoding='utf-8') as csvfile:
#             reader = csv.DictReader(csvfile)
#             for row in reader:
#                 try:
#                     db.insert_restaurant(
#                         nom=row["Nom"],
#                         type_cuisine=row["Type_Cuisine"],
#                         fourchette_prix=row["Fourchette_prix"],
#                         adresse=row["Adresse"],
#                         note_moyenne=row["Note_moyenne"],
#                         description=
#                     )
#                     print(f"Restaurant ajouté : {row['Nom']}")
#                 except KeyError as e:
#                     print(f"Erreur dans le fichier {filename}: champ manquant {e}")


In [10]:
rows_restau = db.fetch_all("SELECT * FROM restaurants")
rows_restau

[(1,
  "L'affreux Jojo",
  'Italienne, Méditerranéenne, Saine',
  '9,00€-15,00€',
  '61 Rue de la Part-Dieu 69003 Lyon France',
  '5,0',
  'Restaurant italien cosy, spécialités : panozzo (pâte à pizza agrémentée de mets frais type charcuteries, fromages, légumes du soleil, crème de truffe), burrata, tiramisu, panacotta, tout est fait Maison. Bellle sélection de vins italiens et français.',
  45.7596229,
  4.8469705),
 (2,
  "L'Argot",
  'Française, Européenne, Saine',
  '15,00€-30,00€',
  '132 Rue Bugeaud 69006 Lyon France',
  '4,5',
  'L\'ARGOT est une Boucherie - Restaurant.\r\n\r\nVous pouvez y venir simplement pour acheter vos viandes et/ou charcuteries ou encore venir déjeuner (du mardi au vendredi inclus) ou dîner (le vendredi et samedi soir).\r\n\r\nLe Restaurant de L\'ARGOT vous propose :\r\n\r\n* Sa "formule du jour" de 12 à 22 euros (uniquement le midi du mardi au vendredi inclus), \r\n* Sa carte, \r\n* Ou, vous laisser tenter par votre viande préférée dans la vitrine du bouc

In [13]:
# Insertion des avis
# test avec affreux_jojo_avis.csv, ID restaurant 1
import csv
def insert_avis(chemin:str, clef:int):
    with open(chemin, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            try:
                db.insert_avis(
                    id_restaurant=clef,
                    date = row["Date"],
                    note=row["Notes"],
                    commentaire=row["Commentaires"]
                )
                # print(f"Avis ajouté : {row['Commentaires']}")
            except KeyError as e:
                print(f"Erreur dans le fichier {filename}: champ manquant {e}")

# insert_avis("../../data/affreux_jojo_avis.csv", 1)
# insert_avis("../../data/chez_michel_avis.csv", 2)
# insert_avis("../../data/chez_michel_avis.csv", 3)
# insert_avis("../../data/chez_michel_avis.csv", 4)
# insert_avis("../../data/chez_michel_avis.csv", 5)
# insert_avis("../../data/chez_michel_avis.csv", 6)
# insert_avis("../../data/chez_michel_avis.csv", 7)
# insert_avis("../../data/chez_michel_avis.csv", 8)
# insert_avis("../../data/chez_michel_avis.csv", 9)
# insert_avis("../../data/chez_michel_avis.csv", 10)

data_directory = "../../data"
cpt = 1
for filename in os.listdir(data_directory):
    filepath = os.path.join(data_directory, filename)

    if filename.endswith("_avis.csv"):
        # print(filename)
        insert_avis(filepath, cpt)
        cpt += 1

Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion réussie.
Insertion ré

In [14]:
rows_avis = db.fetch_all("SELECT * FROM avis")
rows_avis

[(1,
  1,
  'déc. 2024',
  '5,0',
  'Pizza excellente, bonne ambiance, très bon service, au top. On a demandé 2 jojo et 1 pizza et tout a été parfait.\r\n\r\nMerci bcpp!'),
 (2,
  1,
  'déc. 2024',
  '5,0',
  "C'est l'endroit parfait à Lyon pour manger une pizza napolitaine tout à fait délicieuse!\r\nAmbiance et personnel très sympa!"),
 (3,
  1,
  'déc. 2024',
  '5,0',
  'Nous avons dégustés d’excellentes pizze.\r\nUne pizza diavola juste piquante comme il faut et une pizza au jambon truffé, à la crème de truffe parsemé de buratta.\r\nUn repas carrément divin !!!'),
 (4,
  1,
  'nov. 2024',
  '4,0',
  'Tres bel endroit en famille on mange très bien..je conseille...ce restaurant tres belle équipe.et une bonne ambiance'),
 (5,
  1,
  'nov. 2024',
  '5,0',
  'Très bonne pizza bien gourmande et pâtes du jour délicieuse.\r\nDéco et cadre agréable \r\nAccueil très souriant.\r\nJe recommande sans hésiter.'),
 (6,
  1,
  'nov. 2024',
  '5,0',
  "Une pizza ou un panozzo (je ne connaissais pas.

In [9]:
# Select les avis correspondant au restaurant 10
rows_avis_10 = db.fetch_all("SELECT * FROM avis WHERE id_restaurant = 10")
rows_avis_10

[(8731,
  10,
  'déc. 2024',
  '1,0',
  'J’ai procédé à une réservation il y a 3 semaines pour ce samedi 14 décembre 2024 pour le dîner.\r\n\r\nJ’ai alors été mis sur une liste d’attente, par MAIL…et depuis aucun retour!\r\nPas de message pour préciser si notre réservation était validée ou annulée !\r\nRien! \r\nJuste laissé dans l’ignorance !\r\n\r\nC’est juste HONTEUX pour un établissement de ce standing !\r\n\r\nEn commentaire lors de ma réservation j’avais précisé «\xa0dîner pour fêter nos deux anniversaires\xa0» …\r\n\r\nAucun retour, de la part de l’établissement, alors que j’ai donné mes coordonnées, il y avait aussi mon mail!\r\n\r\nLa moindre des POLITESSES - si les personnes de cet établissement savent ce que cela veut ENCORE dire de nos jours, était de mettre un message précisant que notre réservation n’était pas retenue!\r\n\r\nLe flou le plus complet!\r\n\r\nEt dire que vous faites partie des établissements «\xa0Relais & Châteaux\xa0» et bien ce standing a bien changé et v

In [15]:
db.close()

In [1]:
# Test Regex

import re

def extract_postal_code(address: str) -> int:
    """
    Extrait le code postal d'une adresse.

    Args:
        address (str): L'adresse complète.

    Returns:
        int: Le code postal extrait ou None si non trouvé.
    """
    # Regex pattern to match French postal codes (5 digits)
    pattern = r'\b\d{5}\b'
    match = re.search(pattern, address)
    if match:
        return int(match.group(0))
    return None

# Example usage
address = "1 rue de Paris, 75001 Paris"
postal_code = extract_postal_code(address)
print(f"Code postal extrait: {postal_code}")

Code postal extrait: 75001
